In [7]:
import pandas as pd
import numpy as np

import time
import datetime

import folium

import geopandas as gp

from shapely.geometry import Point, Polygon

import random

import h3


In [13]:
Geometry=np.load('./Data/Initialization/Geo.npy',allow_pickle=True).item()

def Check_zones(lat,lng,polys):
    
    pnt=Point(lng,lat)
    
    key=0

    if pnt.within(polys):

        key=1
        
    return key


Maxlng=max(Geometry['Shenzhen'].exterior.coords.xy[0]);Minlng=min(Geometry['Shenzhen'].exterior.coords.xy[0]);

Maxlat=max(Geometry['Shenzhen'].exterior.coords.xy[1]);Minlat=min(Geometry['Shenzhen'].exterior.coords.xy[1]);

def Revise(lng,lat,polys):

    pnt=Point(lng,lat)

    while not pnt.within(polys):

        lng=Minlng+random.random()*(Maxlng-Minlng)
        lat=Minlat+random.random()*(Maxlat-Minlat)
        
        pnt=Point(lng,lat)
        
    return lng,lat

def Verify(lng,lat,polys):
    
    pnt=Point(lng,lat)
    
    if pnt.within(polys):
        return 1
    else:
        return 0

        

resolution=8


# Order data

In [9]:
order_table=pd.read_csv('./Data/Initialization/order_table.csv')

order_table=order_table.drop(columns=['Unnamed: 0'])

order_table['start_grid']=order_table.apply(lambda x:h3.geo_to_h3(x['start_lat'], x['start_lng'], resolution),axis=1)

order_table['end_grid']=order_table.apply(lambda x:h3.geo_to_h3(x['end_lat'], x['end_lng'], resolution),axis=1)

# order_table.to_csv('./Data/Processed/order_table.csv')

# Driver data

In [16]:


Driver_df=pd.read_csv('./Data/Initialization/driver_table.csv')

Driver_df=Driver_df.drop(columns=['Unnamed: 0'])

Driver_df['label']=Driver_df.apply(lambda x:Verify(x['longitude'],x['latitude'],Geometry['Shenzhen']),axis=1)

Driver_df=Driver_df.loc[Driver_df['label']==1]

'''Revise?'''

# Driver_df['coords']=Driver_df.apply(lambda x:Revise(x['longitude'],x['latitude'],Geometry['Shenzhen']),axis=1)

# Driver_df['longitude']=Driver_df.apply(lambda x:x['coords'][0],axis=1)

# Driver_df['latitude']=Driver_df.apply(lambda x:x['coords'][1],axis=1)

Driver_df=Driver_df[['driver_id','longitude','latitude','step']]

Driver_df['grid']=Driver_df.apply(lambda x:h3.geo_to_h3(x['latitude'], x['longitude'], resolution),axis=1)

Driver_df.to_csv('./Data/Processed/driver_table.csv')

Driver_df



,driver_id,longitude,latitude,step,grid
0,5084,113.796811,22.673050,2520,88411c371bfffff
1,2821266,113.894144,22.769028,2520,88411c3181fffff
2,1819797,113.860416,22.660005,2520,88411c3299fffff
5,1881376,114.092919,22.739811,2520,88411c0463fffff
7,2303420,113.921783,22.506436,2520,88411ca193fffff
...,...,...,...,...,...
6040,2569932,113.790535,22.748197,2520,88411c353dfffff
6041,2354955,113.950482,22.569874,2520,88411caf67fffff
6042,2157846,114.406849,22.761615,2520,88411c1841fffff
6043,180386,114.257288,22.740759,2520,88411c0045fffff


In [12]:
'''Background'''

background = folium.Map([22.648883, 114.045011],tiles='CartoDB positron',zoom_start=10)

'''Zones'''

folium.Choropleth(
    geo_data=Geometry['Shenzhen'],
    fill_color='blue',
    fill_opacity=0.0,
    name='Zone').add_to(background)

folium.LayerControl().add_to(background)

Driver_df=pd.read_csv('./Data/Processed/driver_table.csv')
    
for idx,row in Driver_df.iterrows():
    
    folium.Circle(
    radius=10,
    location=[row['latitude'],row['longitude']],
    color='green',
    fill=True,
    ).add_to(background)
        
        

background.save("Driver_distribution.html")